get a list of folder topics from bookmarks?


In [ ]:
var _ = require('lodash');
var importer = require('../Core');
var getBookmarksFromTakeout = importer.import('parse bookmarks file');

function getBookmarkFolders() {
    return getBookmarksFromTakeout()
        .then(links => {
            // group by half hour
            const groups = Object.keys(links).sort((a, b) => parseInt(b) - parseInt(a));
            return groups.reduce((acc, t) => {
                const folders = _.groupBy(links[t], l => l.folder);
                acc = acc.concat(Object.keys(folders));
                return acc;
            }, []);
        })
        .then(folders => folders.filter((f, i, arr) => arr.indexOf(f) === i))
}

module.exports = getBookmarkFolders;

if(typeof $$ != 'undefined') {
    $$.async();
    
    getBookmarkFolders()
        .then(r => $$.mime({'text/plain': r.join('\n')}))
        .catch(e => $$.sendError(e))
}


update insterests resume sheet?


In [ ]:
var _ = require('lodash');
var importer = require('../Core');
var getBookmarksFromTakeout = importer.import('parse bookmarks file');
var updateRow = importer.import('update a row in google sheets')
var getDataSheet = importer.import('get google data sheet')

var doc = 'https://docs.google.com/spreadsheets/d/1dAtBQyn5vwStM6ITn7iCpp996Sus26k0bJtbGNlCX2g/edit#gid=257854275';

function updateInterestPage() {
    var links, interests;
    return getDataSheet(doc, 'Interests data')
        .then(rows => interests = rows.map(r => r.category).filter((r, i, arr) => arr.indexOf(r) === i))
        .then(() => getBookmarksFromTakeout())
        .then(l => links = [].concat(...Object.values(l))
              .filter(b => interests.includes(b.folder))
              .map(b => ({
                category: b.folder,
                date: Math.round(b.time_usec / 1000 / 10000) + '',
                title: b.title,
                link: b.url,
                skills: null,
                tools: null
            })))
        .then(() => importer.runAllPromises(links.map(l => resolve => {
            // category date title link skills tools
            return updateRow(doc, r => {
                debugger;
                return r.category == l.category && !r.link
                             || r.date == l.date && r.link == l.link}, l, 'Interests data')
                .then(resolve);
        })))
}

module.exports = updateInterestPage;

if(typeof $$ != 'undefined') {
    $$.async();
    
    updateInterestPage()
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}